In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp


In [5]:

r1, r2, r3, r4, a1, a2 = 0, 0.400, 0.455, 10**(-3)*np.sqrt(420**2+35**2), 0.025, 0.035
køddevinkel = np.arctan2(35, 420)


T_SB = np.array([[0,  0, 1, 0.525],
                 [0, -1, 0, 0],
                 [1,  0, 0, 0.89],
                 [0,  0, 0, 1]])




sp.Matrix(np.around(tools.inverseKinematicsTheta123(T_SB),7))


[0.445 0.    0.89 ]
0.08304547985373995


Matrix([
[    0,   8.7974107,  85.2363583],
[    0,        90.0, -85.2363583],
[180.0, 171.2025893,  85.2363583],
[180.0,        90.0, -85.2363583]])

In [3]:

config_fasit = np.array([0,-np.pi/4, np.pi/6, 0,0,0])

config2 = np.array([0, -np.pi/2, np.pi/2, 0, 0, 0])
config3 = np.array([0, np.pi/2, 0, 0, 0, 0])




def test_a_IK(config_fasit):

    kr6_fasit = KR6(config_fasit)

    P_SW_fasit = tools.pointFromT(kr6_fasit.Tsw) # define the point of the wrist

    thetalist = kr6_fasit.aIK_thetaLists # Fetching the analytical solution(s)

    for el in thetalist:
        config_test = np.array([el[0], el[1], el[2], 0,0,0])

        kr6_test = KR6(config_test)

        P_SW_test = tools.pointFromT(kr6_test.Tsw)

        print(kr6_fasit.Tsb_poe)
        # print(kr6_fasit.Tsw)
        print(P_SW_test)
        print(P_SW_fasit)

        print(np.allclose(P_SW_fasit, P_SW_test, 0.01, 0.01))

print(test_a_IK(np.array([0, 0, 0, 0, 0, 0])))

# kr6.add_FK_to_viewer(viewer)

# print(f"Number of iteration: {kr6.IK[3]}")

# kr6.add_nIK_to_viewer(viewer)
viewer = Viewer(background="white")
viewer.add(Grid())
KR6(config_fasit).add_FK_to_viewer(viewer)


[9.00000000e-01 3.94948593e-17 4.35000000e-01]
0.9965457582448798
[4.66563035e-01 1.28353537e-15 8.21241167e-01]
-0.03188311736686809
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[0.46656233 0.         0.82124157]
[0.9   0.    0.435]
False
[3.72170796e-01 1.07200551e-15 9.61703504e-01]
0.13398974287493645
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[0.3721708 0.        0.9617035]
[0.9   0.    0.435]
False
[ 0.34515376 -0.46205396  0.50166234]
-0.08574162905956745
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]
[ 0.34515376 -0.46205396  0.50166234]
[0.9   0.    0.435]
False
[-0.40415208  0.54103443  0.28544767]
0.24700751198530518
[[ 0.     0.     1.     0.98 ]
 [ 0.    -1.     0.     0.   ]
 [ 1.     0.     0.     0.435]
 [ 0.     0.     0.     1.   ]]

C:\Users\test\anaconda3\envs\tpk4170_venv\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

[7.43363766e-01 1.18345884e-16 8.64244988e-01]
0.904556682600562
